In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

## Настройка браузера

In [ ]:
options = Options()
options.add_argument("--headless=new")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

driver.get("https://www.tbank.ru/invest/social/profile/T-Trading/?author=profile")
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "[data-qa-tag='PulsePost']")))

# Параметры сбора
max_posts = 30
all_posts = {}
no_increase = 0
max_no_increase = 3
last_unique = 0
stop_early = False

<selenium.webdriver.remote.webelement.WebElement (session="86e0b13b059651c5970f38b2c75a7ac3", element="96F314EB2442A69AAF5CBEEB12D6F7F6_element_42")>

## Сбор постов

In [ ]:
print("Загрузка постов...\n")

# Скролл и сбор постов
while no_increase < max_no_increase:
    post_containers = driver.find_elements(By.CSS_SELECTOR, "[data-qa-tag='PulsePost']")

    for container in post_containers:
        try:
            text = container.text.strip()
            if not text:
                continue
            key = text[:100] # Защита от дубликатов по первым 100 символам

            if len(all_posts) >= max_posts:
                stop_early = True
                break

            if key not in all_posts:
                # Дата публикации
                try:
                    date_elem = container.find_element(By.CSS_SELECTOR, "[data-qa-file='PulsePostAuthor'] .social-profile__cSULlZ")
                    date_text = date_elem.text
                except:
                    date_text = ""

                # HTML содержимого поста
                try:
                    body = container.find_element(By.CSS_SELECTOR, "[data-qa-tag='PulsePostBody']")
                    try:
                        btn = body.find_element(By.XPATH, ".//button[contains(., 'Читать дальше')]")
                        driver.execute_script("arguments[0].click();", btn)
                        time.sleep(0.3)
                    except:
                        pass
                    try:
                        content = body.find_element(By.CSS_SELECTOR, "[data-qa-file='TextLineCollapse']")
                        html = content.get_attribute("innerHTML")
                    except:
                        html = body.get_attribute("innerHTML")
                except:
                    html = ""

                all_posts[key] = {"date_raw": date_text, "html": html, "text": text}
        except:
            pass

    if stop_early:
        print(f"Достигнут лимит в {max_posts} постов")
        break

    current_unique = len(all_posts)
    print(f"Собрано постов: {current_unique}")

    # Скроллим к последнему видимому посту
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", post_containers[-1])
    time.sleep(1)

    if current_unique > last_unique:
        last_unique = current_unique
        no_increase = 0
    else:
        no_increase += 1
        if no_increase >= max_no_increase:
            print("Достигнут лимит попыток")
            break

driver.quit()
print(f"\nИтого постов собрано: {len(all_posts)}")


Загрузка постов...

Достигнут лимит в 30 постов.

Итого постов собрано: 30
Строк в датафрейме: 196


## Функции парсинга и преобразования даты

In [ ]:
def parse_publication_date(date_raw, now=None):
    """Преобразует строку даты поста в формат YYYY-MM-DD"""
    if now is None:
        now = pd.Timestamp.now()
    date_part = date_raw.split(" в ")[0].strip() if " в " in date_raw else date_raw.strip()
    date_part_lower = date_part.lower()

    if date_part_lower == "вчера":
        return (now - pd.Timedelta(days=1)).strftime("%Y-%m-%d")
    if date_part_lower in ("сегодня", "только что"):
        return now.strftime("%Y-%m-%d")

    months = {
        'января': '01', 'февраля': '02', 'марта': '03', 'апреля': '04',
        'мая': '05', 'июня': '06', 'июля': '07', 'августа': '08',
        'сентября': '09', 'октября': '10', 'ноября': '11', 'декабря': '12',
    }
    for rus, num in months.items():
        if rus in date_part_lower:
            date_part = re.sub(rus, num, date_part, flags=re.IGNORECASE)
            break

    parts = date_part.split()
    try:
        if len(parts) == 2:
            return f"{now.year:04d}-{int(parts[1]):02d}-{int(parts[0]):02d}"
        if len(parts) == 3:
            return f"{int(parts[2]):04d}-{int(parts[1]):02d}-{int(parts[0]):02d}"
    except:
        pass
    return now.strftime("%Y-%m-%d")


def extract_forecast_date(text, pub_date):
    """Извлекает дату торгов из текста поста"""
    match = re.search(r"по итогам торгов\s+(\d{1,2})\s+([а-яА-Я]+)", text)
    if not match:
        return pub_date
    months = {
        'января': 1, 'февраля': 2, 'марта': 3, 'апреля': 4,
        'мая': 5, 'июня': 6, 'июля': 7, 'августа': 8,
        'сентября': 9, 'октября': 10, 'ноября': 11, 'декабря': 12,
    }
    month = months.get(match.group(2).lower())
    if not month:
        return pub_date
    try:
        year = pd.to_datetime(pub_date).year
        return f"{year:04d}-{month:02d}-{int(match.group(1)):02d}"
    except:
        return pub_date


def parse_block(block_text, publication_date, last_trade_date, direction):
    """Парсит один блок бумаги и возвращает словарь с данными прогноза"""
    ticker_match = re.search(r"\$(\w+)", block_text)
    potential_match = re.search(r"Потенциал (?:роста|падения) ([\d\.]+)%", block_text)
    horizon_match = re.search(r"Срок (\d+) (?:день|дня|дней)", block_text)

    if not all([ticker_match, potential_match, horizon_match]):
        return None

    potential = float(potential_match.group(1))
    if direction == "down":
        potential = -potential
    horizon = int(horizon_match.group(1))
    name = block_text.strip().split("\n")[0].strip()

    return {
        "publication_date": publication_date,
        "last_trade_date": last_trade_date,
        "ticker": ticker_match.group(1),
        "name": name,
        "potential": potential,
        "forecast_days": horizon,
        "forecast_end_date": (pd.to_datetime(publication_date) + pd.Timedelta(days=horizon)).strftime("%Y-%m-%d")
    }


def parse_post_html(html, publication_date, last_trade_date):
    """Парсит HTML поста и возвращает список прогнозов"""
    text = BeautifulSoup(html, "html.parser").get_text("\n")
    if "Лучшие торговые прогнозы по теханализу" not in text:
        return []

    rows = []
    gpos, dpos = text.find("📈"), text.find("📉")

    if gpos != -1:
        end = dpos if (dpos != -1 and dpos > gpos) else len(text)
        for b in re.split(r"💼\s*Бумага", text[gpos:end])[1:]:
            parsed = parse_block(b, publication_date, last_trade_date, "up")
            if parsed:
                rows.append(parsed)

    if dpos != -1:
        for b in re.split(r"💼\s*Бумага", text[dpos:])[1:]:
            parsed = parse_block(b, publication_date, last_trade_date, "down")
            if parsed:
                rows.append(parsed)

    return rows

In [ ]:
# Сборка датафрейма
all_rows = []
for item in all_posts.values():
    pub_date = parse_publication_date(item['date_raw'])
    trade_date = extract_forecast_date(item['text'], pub_date)
    all_rows.extend(parse_post_html(item['html'], pub_date, trade_date))

df = pd.DataFrame(all_rows)
print(f"Строк в датафрейме: {df.shape[0]}")

# df.to_csv("tinvest_forecasts.csv", index=False, encoding="utf-8-sig")